In [12]:
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from umap.umap_ import UMAP
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import os
import scanpy as sc

import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mse
from sklearn.utils import resample
import lightgbm as lgb
import shap

In [2]:
adata = sc.read_h5ad("mouse_diabetes.h5ad", backed='r')

# Access obs (cell metadata) - this is fully loaded into memory
obs_df = adata.obs.copy()
obs_df = obs_df[~obs_df['cell_type'].isin(['unknown', 'cell'])]

# Determine total samples
total_sample_size = 20000
np.random.seed(42)


# Optional: check your metadata structure
# print(obs_df['cell_type'].value_counts())
# print(obs_df['disease_type'].value_counts())

# Create a combined stratification column if you want to conserve both cell type and disease type
obs_df['strata'] = obs_df['cell_type'].astype(str) + "_" + obs_df['disease'].astype(str)

# Sample size per strata proportional to their representation
strata_counts = obs_df['strata'].value_counts()
strata_sample_sizes = (strata_counts / strata_counts.sum() * total_sample_size).round().astype(int)

# Perform sampling per strata
sampled_indices = []
for strata_value, sample_size in strata_sample_sizes.items():
    strata_indices = obs_df[obs_df['strata'] == strata_value].index
    if len(strata_indices) < sample_size:
        sample_size = len(strata_indices)  # avoid over-sampling
    sampled_indices.extend(np.random.choice(strata_indices, sample_size, replace=False))

# Now, create a new AnnData with only these samples
adata_sampled = adata[sampled_indices].to_memory()

#sc.pp.log1p(adata_sampled)

In [5]:
if not isinstance(adata_sampled.X, np.ndarray):
    X = adata_sampled.X.toarray()  # convert sparse to dense
else:
    X = adata_sampled.X

# Optional: clean data
X = np.nan_to_num(X)

# Labels
y = adata_sampled.obs["disease"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
clf = lgb.LGBMClassifier(
    boosting_type='gbdt',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=-1,
    random_state=42
)
clf.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.697482 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3195973
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 17192
[LightGBM] [Info] Start training from score -2.269997
[LightGBM] [Info] Start training from score -0.994083
[LightGBM] [Info] Start training from score -1.890062
[LightGBM] [Info] Start training from score -0.979330


LGBMClassifier(random_state=42)

In [45]:
gene_names = adata_sampled.var_names.tolist()

# Subsample X and y for SHAP to ~1000-2000 cells
#X_sample = X[:2000]
#y_sample = y[:2000]
X_sample, y_sample = resample(X, y, n_samples=2000, random_state=42, stratify=y)

explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_sample)





In [62]:
print(shap_values.shape)  # Check shape of SHAP values for the first class

abs_shap = np.abs(shap_values)
print(abs_shap.shape)  # Should be (n_samples, n_classes, n_genes)

# Mean across cells and classes → shape will be (genes,)
mean_abs_shap = abs_shap.mean(axis=(0, 2))


print(mean_abs_shap.shape)  # Should be (genes,)

top_1000_idx = np.argsort(mean_abs_shap)[-1000:][::-1]
print(top_1000_idx.shape)  # Should be (1000,)

top_1000_genes = [gene_names[i] for i in top_1000_idx]


(2000, 31202, 4)
(2000, 31202, 4)
(31202,)
(1000,)


In [70]:
with open("top_1000_genes.txt", "w") as f:
    for gene in top_1000_genes:
        f.write(gene + "\n")